In [1]:
import numpy as np
import sympy as sp
import pandas as pd
from numpy.linalg import norm

In [2]:
def levenberg_marquardt(func, x, t, t_vec, y_vec, x_0, tol = 1e-4, max_iter = 1e3, lambda_0 = 10):
    
    # Definimos la función, los residuos y el san jacobiano
    f = [func.subs({t: t_j}) for t_j in t_vec]
    r = [f_j - y_j for f_j, y_j in zip(f, y_vec)]
    jacobo = sp.Matrix(f).jacobian([x])
    
    results = []
    x_k = x_0
    lambda_k = lambda_0
    diff_cond = 2 * tol
    iter_count = 0
    last_iter = False
    while iter_count < max_iter:

        jacobo_k = jacobo.subs({x: x_k})
        r_k = np.asarray([r_j.subs({x: x_k}) for r_j in r], dtype = np.float16)
        d_k = - (jacobo_k.T.dot(jacobo_k) + lambda_k * jacobo_k.T.dot(jacobo_k))**(-1) * jacobo_k.T.dot(r_k)
        
        x_k1 = x_k + d_k
        r_k1 = np.asarray([r_j.subs({x: x_k1}) for r_j in r], dtype = np.float16)

        lambda_cond = norm(r_k1) ** 2 - norm(r_k)**2
        results.append([iter_count, x_k, sum(r_k**2), lambda_k, d_k])

        if lambda_cond > 0:
            lambda_k *= 10
        else:
            lambda_k /= 10
            x_k = x_k1
            diff_cond = lambda_cond
        iter_count += 1

        if last_iter:
            break
        if abs(diff_cond) < tol:
            last_iter = True

    table = pd.DataFrame(results)
    table.columns = ['Iter', 'x', 'Suma(r^2)', 'lambda', 'd']
    return table, x_k

In [3]:
# Datos
t_vec = [0, 0.5, 1, 1.5, 2, 2.5, 3]
y_vec = [1.145, 0.512, 0.401, 0.054, 0.038, 0.014, 0.046]
x_0 = 4

# Ecuación de ajuste
x, t = sp.symbols('x t')
f = np.e ** (- x * t)

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(f, x, t, t_vec, y_vec, x_0)
results_table

,Iter,x,Suma(r^2),lambda,d
0,0,4,0.315738,10.000,-0.603468900142443
1,1,3.39653109985756,0.270327,1.000,-2.24095234867595
2,2,1.15557875118161,0.051601,0.100,0.102895790603077
3,3,1.25847454178469,0.047835,0.010,0.0117833752674878
4,4,1.27025791705218,0.047798,0.001,-0.000502339975430241
